In [1]:
import pandas as pd
import nltk as nltk
from emoji import UNICODE_EMOJI
import unicodedata as unicodedata
import re
from __future__ import print_function
import os  # for os.path.basename
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
respuestas=pd.read_excel('/Users/pepe_opi/Downloads/results_export_964.xlsx')

In [4]:
respuestas.head()

,Contact UUID,URN,Name,Groups,Response 9 (Category) - uncaught,Response 9 (Value) - uncaught,Response 9 (Text) - uncaught,Response 8 (Category) - uncaught,Response 8 (Value) - uncaught,Response 8 (Text) - uncaught,Response 10 (Category) - uncaught,Response 10 (Value) - uncaught,Response 10 (Text) - uncaught,Response 9 (Category) - uncaught.1,Response 9 (Value) - uncaught.1,Response 9 (Text) - uncaught.1,Response 11 (Category) - uncaught,Response 11 (Value) - uncaught,Response 11 (Text) - uncaught,Response 10 (Category) - uncaught.1,Response 10 (Value) - uncaught.1,Response 10 (Text) - uncaught.1,menu (Category) - uncaught,menu (Value) - uncaught,menu (Text) - uncaught,Response 11 (Category) - uncaught.1,Response 11 (Value) - uncaught.1,Response 11 (Text) - uncaught.1,respuesta_pregs (Category) - uncaught,respuesta_pregs (Value) - uncaught,respuesta_pregs (Text) - uncaught,menu_pd (Category) - uncaught,menu_pd (Value) - uncaught,menu_pd (Text) - uncaught,Response 12 (Category) - uncaught,Response 12 (Value) - uncaught,Response 12 (Text) - uncaught
0,3bf0c4bb-d5db-4e0d-ab29-98a7ee6b2b6a,1104797289620477,NaN,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...,Other,NaN,La solicitud de enviar el SMS por Cobrar a 552...,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...,No,no,Tu mensaje ha excedido el tiempo de respuesta ...,NaN,NaN,NaN,NaN,NaN,NaN,No,no,Tu mensaje ha excedido el tiempo de respuesta ...,NaN,NaN,NaN,NaN,NaN,NaN,seguridad,2,Tienes un SMSxCobrar esperando ser enviado a $...,NaN,NaN,NaN
2,7ea8e04b-e8d0-4628-bf35-350a784d5e2f,+525559652925,pedro vallejo,"ALL, ALTOPD",Emoji,🙅,🙅,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3059e012-6fd3-445d-8da4-5fc36d45f8dc,1326847360726593,NaN,ALTOPD,NaN,NaN,NaN,NaN,NaN,NaN,Other,Ya no quiero recibir mensajes,Ya no quiero recibir mensajes,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42bf4e3f-a396-4eb7-a987-b76cba779e92,+527711780721,Rosy,"ALL, ALTOPD",Other,Hola gordo,Hola gordo,con_nombre,Rosy,Hola gordo,Other,Hola gordo,Hola gordo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
respuestas=respuestas[['Contact UUID', 'URN', 'Name', 'Groups',
       'Response 9 (Category) - uncaught', 'Response 9 (Value) - uncaught',
       'Response 9 (Text) - uncaught']]

respuestas.columns=['id_contacto', 'urn', 'nombre', 'listas', 'uncaught_resp_9_cat',
                    'uncaught_resp_9_val', 'uncaught_resp_9_text']

In [6]:
respuestas.head()

,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text
0,3bf0c4bb-d5db-4e0d-ab29-98a7ee6b2b6a,1104797289620477,NaN,"ALL, PUERPERIUM, PUERPERIUM_MS, PUERPERIUM_MS_...",NaN,NaN,NaN
1,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,La solicitud de enviar el SMS por Cobrar a 552...
2,7ea8e04b-e8d0-4628-bf35-350a784d5e2f,+525559652925,pedro vallejo,"ALL, ALTOPD",Emoji,🙅,🙅
3,3059e012-6fd3-445d-8da4-5fc36d45f8dc,1326847360726593,NaN,ALTOPD,NaN,NaN,NaN
4,42bf4e3f-a396-4eb7-a987-b76cba779e92,+527711780721,Rosy,"ALL, ALTOPD",Other,Hola gordo,Hola gordo


In [7]:
respuestas['uncaught_resp_9_cat'].value_counts()

Other            954
Like-fb          452
si               312
Claro            226
no               153
gracias          136
Ok                98
Emoji             56
Buenas noches     39
Gracias           35
Hasta luego        6
Twitter image      5
ok                 1
Name: uncaught_resp_9_cat, dtype: int64

In [8]:
respuestas.loc[respuestas['uncaught_resp_9_cat']=='Twitter image', 'uncaught_resp_9_text'].value_counts()

https://t.co/az6m6XXjoL     1
https://t.co/rbsa5QOafg     1
https://t.co/HouKMudLQc     1
 https://t.co/8kYuMPNM3V    1
https://t.co/Waly9IfMui     1
Name: uncaught_resp_9_text, dtype: int64

In [9]:
respuestas.loc[respuestas['uncaught_resp_9_cat']=='Hasta luego', 'uncaught_resp_9_text'].value_counts()

Hasta luego            2
Hasta luego gracias    1
hasta luego!           1
Hasta luego!           1
HASTA LUEGO            1
Name: uncaught_resp_9_text, dtype: int64

In [10]:
respuestas['uncaught_resp_9_cat'].value_counts()

Other            954
Like-fb          452
si               312
Claro            226
no               153
gracias          136
Ok                98
Emoji             56
Buenas noches     39
Gracias           35
Hasta luego        6
Twitter image      5
ok                 1
Name: uncaught_resp_9_cat, dtype: int64

## preprocesamiento

In [11]:
respuestas['categ_opi']=None

In [12]:
respuestas['categ_opi'].loc[respuestas['uncaught_resp_9_cat'].isin(list(['Like-fb', 'Twitter image','Hasta luego']))]=respuestas['uncaught_resp_9_cat'].loc[respuestas['uncaught_resp_9_cat'].isin(list(['Like-fb', 'Twitter image','Hasta luego']))]

In [13]:
nulos=respuestas.loc[respuestas.uncaught_resp_9_text.isnull()]
respuestas=respuestas.loc[respuestas.uncaught_resp_9_text.notnull()]

### Pasar a minúsculas

In [14]:
respuestas['uncaught_resp_9_text']=respuestas['uncaught_resp_9_text'].str.lower()

 ### Quitar leading y trailing spaces, quitar puntuacion

In [15]:
respuestas['uncaught_resp_9_text']=respuestas['uncaught_resp_9_text'].str.strip()

In [16]:
respuestas['uncaught_resp_9_text_wc'] = respuestas['uncaught_resp_9_text'].apply(lambda x: len(str(x).split(" ")))
respuestas['uncaught_resp_9_text_nchar'] = respuestas['uncaught_resp_9_text'].apply(lambda x: len(str(x)))

In [17]:
respuestas['categ_opi'].loc[(respuestas['uncaught_resp_9_text'].str.count('\\?')==respuestas['uncaught_resp_9_text_nchar'])]='duda'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text'].str.replace('[^\w\s]','')

In [19]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text_mod'].str.strip()

In [20]:
respuestas['uncaught_resp_9_text_wc'] = respuestas['uncaught_resp_9_text_mod'].apply(lambda x: len(str(x).split(" ")))
respuestas['uncaught_resp_9_text_nchar'] = respuestas['uncaught_resp_9_text_mod'].apply(lambda x: len(str(x)))

### Capturar emojis

Actualmente, si el mensaje contiene un emoji, es canalizado a la categoría Emoji. Este filtro debe ser más estricto, pues usualmente ese emoji es usado en un mensaje real

In [21]:
def is_emoji(s):
    count = 0
    for emoji in range(1, len(UNICODE_EMOJI)):
        count += s.count(list(UNICODE_EMOJI.keys())[emoji])
    return count

In [22]:
es_emoji=pd.DataFrame()
for i in range(0, respuestas.shape[0]):
    es=pd.DataFrame({'es':is_emoji(respuestas['uncaught_resp_9_text'].iloc[i])}, index=[0])
    es_emoji=pd.concat([es_emoji, es], ignore_index=True)

In [23]:
respuestas=respuestas.reset_index(drop=True)

In [24]:
respuestas=pd.concat([respuestas, es_emoji], axis=1)

In [25]:
respuestas['categ_opi'].loc[(respuestas.es>0) &
                            (respuestas.uncaught_resp_9_text_mod=='')]='emoji'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Quitar emojis que quedan en el texto

In [26]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [27]:
for i in range(0, respuestas.shape[0]):
    respuestas['uncaught_resp_9_text_mod'].iloc[i]=give_emoji_free_text(respuestas.uncaught_resp_9_text_mod.iloc[i])

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Quitar  acentos

In [28]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('https')) &
              (respuestas.uncaught_resp_9_text_wc==1) & (respuestas.categ_opi.isnull())]='spam'



/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
for i in range(0, respuestas.shape[0]):
    respuestas['uncaught_resp_9_text_mod'].iloc[i]=unicodedata.normalize('NFD', respuestas['uncaught_resp_9_text_mod'].iloc[i]).encode('ascii', 'ignore').decode('utf-8')

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('si|sii')) &
                            (respuestas.uncaught_resp_9_text_nchar==2)]='si'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('no|noo')) &
                            (respuestas.uncaught_resp_9_text_nchar==2)]='no'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('gracias')) &
               (respuestas.uncaught_resp_9_text_wc==1) ]='gracias'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('ok|ook|okk')) & (respuestas.uncaught_resp_9_text_wc==1) &
              (respuestas['categ_opi'].isnull())]='ok'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('hola')) &
              (respuestas.uncaught_resp_9_text_wc<=2) &
              (respuestas.categ_opi.isnull())]='hola'
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('hola|buenas tardes|buenas noches|buenos dias')) &
              (respuestas.uncaught_resp_9_text_wc<=4) &
              (respuestas.categ_opi.isnull())]='hola'
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('gracias')) &
              (respuestas.uncaught_resp_9_text_wc<=5) &
              (respuestas.categ_opi.isnull())]='gracias'
respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('horario')) &
               (respuestas.uncaught_resp_9_text_mod.str.contains('atencion')) &
              (respuestas.categ_opi.isnull())]='pregunta_horario'

respuestas['categ_opi'].loc[(respuestas.uncaught_resp_9_text_mod.str.contains('numero')) &
               (respuestas.uncaught_resp_9_text_mod.str.contains('telefono')) &
               (respuestas.uncaught_resp_9_text_wc<10) &
              (respuestas.categ_opi.isnull())]='pregunta_telefono'

respuestas.categ_opi.loc[respuestas.uncaught_resp_9_text_mod.str.contains('added') &
              respuestas.categ_opi.isnull()]='spam'

/Users/pepe_opi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
respuestas.loc[respuestas.categ_opi.isnull()].to_csv('etiquetado.csv')

In [38]:
respuestas.loc[respuestas.categ_opi.isnull()]

,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text,categ_opi,uncaught_resp_9_text_wc,uncaught_resp_9_text_nchar,uncaught_resp_9_text_mod,es
0,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,la solicitud de enviar el sms por cobrar a 552...,None,24,144,la solicitud de enviar el sms por cobrar a 552...,0
10,313822ff-7541-4db6-8fe1-a252363000ab,+522461582793,sarahi,"ALL, ALTOPD",no,no,te llame y no pude localizarte. tramita en lin...,None,24,135,te llame y no pude localizarte tramita en line...,0
16,86962a26-c021-4606-8986-893834ad83bd,+524271210385,NaN,"ALL, ALTOPD",Other,59508,59508,None,1,5,59508,0
17,84b7624f-261d-44b4-83ba-6fade1a44c3a,+524271654479,NaN,"ALL, ALTOPD",Other,Mi bebe,mi bebe,None,2,7,mi bebe,0
29,e083b2ea-6e2d-410c-b5a4-8067a818d485,+522411259330,NaN,"ALL, ALTOPD",Gracias,gracias,hola si ya cada integrante tenemos fotos con l...,None,18,99,hola si ya cada integrante tenemos fotos con l...,0
33,b075e779-9037-48c3-80e2-7dbe8eaf399d,520342471032511,NaN,NaN,si,SI,el usuario 2471032511 te envio un mensaje pero...,None,17,99,el usuario 2471032511 te envio un mensaje pero...,0
36,df6ae52e-ef01-46d3-8837-f8dc436b4680,+522461618089,NaN,"ALL, ALTOPD",no,no,te llame y no pude localizarte. ¡ultima oportu...,None,19,129,te llame y no pude localizarte ultima oportuni...,0
37,e9f87b85-422f-4db0-9434-435e51c80c85,+522471337382,NaN,"ALL, ALTOPD",no,no,"te llame y no pude localizarte. 3,000 mxn en m...",None,21,135,te llame y no pude localizarte 3000 mxn en mon...,0
38,a6810ce3-f7da-4d0d-92c5-f35c38d729dd,+522227091226,NaN,"ALL, ALTOPD",Other,Mi bebé esta deshidratado,mi bebé esta deshidratado,None,4,25,mi bebe esta deshidratado,0
39,a78fe386-b115-410b-911c-5a957e0da2d9,812764035515292,Miguel Vilchis,"ALL, ALTOPD",Other,mensaje fuera de flujo,mensaje fuera de flujo,None,4,22,mensaje fuera de flujo,0


In [40]:
x_train=pd.read_csv('/Users/pepe_opi/Documents/GIT/mi-salud/datos/etiquetado_pepe.csv')

In [41]:
#x_train.reset_index(drop=True).sample(100).to_csv('muestra_1')
#x_train.reset_index(drop=True).sample(100).to_csv('muestra_2')

In [42]:
x_target=x_train.categ_opi
x_train=x_train.uncaught_resp_9_text

In [43]:
stop=nltk.corpus.stopwords.words("spanish")

In [44]:
stemmer = SnowballStemmer("spanish")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [45]:
totalvocab_stemmed = []
totalvocab_tokenized = []
lista_fin=list()
for i in x_train:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [46]:

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,2))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(x_train) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 1.47 s, sys: 22.3 ms, total: 1.5 s
Wall time: 1.53 s
(993, 17102)


In [47]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(tfidf_matrix, x_target)



In [48]:
test=respuestas.uncaught_resp_9_text_mod.loc[respuestas.categ_opi.isnull()]


In [49]:
tfidf_matrix_test = tfidf_vectorizer.transform(test)



In [50]:
pd.DataFrame(clf.predict(tfidf_matrix_test))[0].value_counts()



5    911
6     75
8      6
7      1
Name: 0, dtype: int64

In [53]:
test.loc[clf.predict(tfidf_matrix_test)==7]

408    bolsa de trabajo
Name: uncaught_resp_9_text_mod, dtype: object

In [310]:
terms = tfidf_vectorizer.get_feature_names()